In [1]:
using CSV
using DataFrames
using Dates
using ShiftedArrays

df = CSV.read("bloomberg_index.csv", DataFrame)
hy_df = df[:,[:Date, :US_High_Yield]]
dropmissing!(hy_df)
hy_df[!,"lag"] = mapcols(lag, hy_df)[!,"US_High_Yield"]
num_df = hy_df[:,["US_High_Yield","lag"]]
mapcols!(x -> log.(x), num_df)
hy_df[!,"return"] = num_df[!,"US_High_Yield"] - num_df[!,"lag"]
hy_df.Date = Date.(hy_df.Date,"y/m/d")

factor_df = CSV.read("FI400L_FacRet.20210115", DataFrame, delim="|")
factor_df[:,"Date"] = string.(factor_df[:,"DataDate"])
factor_set = Set{String}(factor_df[!,:Factor])

da = "df_test = DataFrame(Date=String[],"
for item in factor_set
    global da
    da *= (item * "=Float64[],")
end
da *= ")"
eval(Meta.parse(da))

date_set = Set{String}(factor_df[!,:Date])
date_array = []
for item in date_set
    push!(date_array,item)
end

for item in date_set
    push!(df_test, Dict(:Date=>item), cols=:union)
end

i = 1
for i in 1:length(factor_df[!,:Date])
    df_test[df_test.Date.==factor_df[i,:Date],factor_df[i,:Factor]] .= factor_df[i,:Return]
end

df_test.Date = Date.(df_test.Date,"yyyymmdd")
sort!(df_test,:Date)
data_set = innerjoin(df_test,hy_df[!,["Date","return"]],on=:Date)
data_set |> CSV.write("data_set.csv",delim=',',writeheader=true)

us_factor_list = String[]
for s in factor_set
    if occursin(r"^US", s) | occursin(r"^JP", s) | occursin(r"^CAD", s) | occursin(r"^CHN", s) | occursin(r"^EM", s) | occursin(r"^EU", s) | occursin(r"^GBP", s)
        push!(us_factor_list,s)
    end
end

In [2]:
push!(us_factor_list,"Date")
push!(us_factor_list,"return")

426-element Array{String,1}:
 "USD_MBS_ALL"
 "USD_MBS_CONVENTIONAL_15YR"
 "US_CDT_DTS_COND_MEDIA_IG"
 "EUR_SOV_SPREAD_IRL_20Y"
 "EUR_SOV_SPREAD_PRT_30Y"
 "EUR_EMU_INFL_50Y"
 "EUR_SOV_SPREAD_NLD_10Y"
 "US_CDT_DTS_COND_HY"
 "EUR_SOV_SPREAD_ITA_2Y"
 "JPY_GOV_20Y"
 "EU_CDT_DTS_COND"
 "GBP_SWAP_SPREAD_5Y"
 "USD_BASIS_OTR_1Y"
 ⋮
 "EUR_SOV_SPREAD_ESP_20Y"
 "CAD_INFL_50Y"
 "US_CDT_DTS_MAT_MET_IG"
 "EM_CORP_DTS_EUROPE_FIN_IG"
 "EUR_SOV_SPREAD_FRA_30Y"
 "EUR_SOV_SPREAD_ESP_2Y"
 "USD_LOAN_HLTH"
 "JPY_IMPVOL_IMPLIED_VOL"
 "USD_SWAP_SPREAD_20Y"
 "US_CDT_DTS_IND_HY"
 "Date"
 "return"

In [3]:
us_data_set = data_set[!,us_factor_list]

1144×426 DataFrame
  Row │ USD_MBS_ALL        USD_MBS_CONVENTIONAL_15YR  US_CDT_DTS_COND_MEDIA_IG ⋯
      │ Float64?           Union{Missing, Float64}    Union{Missing, Float64}  ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ missing                         missing               missing          ⋯
    2 │ missing                         missing               missing
    3 │ missing                         missing               missing
    4 │ missing                         missing               missing
    5 │ missing                         missing               missing          ⋯
    6 │ missing                         missing               missing
    7 │ missing                         missing               missing
    8 │ missing                         missing               missing
    9 │ missing                         missing               missing          ⋯
   10 │ missing                         missing               missing
   11 │ missing                         missing               missing
  ⋮   │         ⋮                      ⋮                         ⋮             ⋱
 1135 │       0.0003933                       0.0002               -0.0312747
 1136 │      -0.000162249                     0.0002                0.0572448  ⋯
 1137 │       0.00185848                      0.0005                0.00847769
 1138 │       0.000395103                     0.0002                0.0304442
 1139 │       6.15988e-5                     -0.0003                0.0321978
 1140 │       0.000291039                     0.0007                0.0606491  ⋯
 1141 │       3.32135e-5                      0.0002               -0.0525921
 1142 │       0.000551678                     0.0004                0.042009
 1143 │       0.000256595                     0.001                -0.0444982
 1144 │       0.000465226                     0.0003                0.0182466  ⋯
                                               423 columns and 1123 rows omitted

In [4]:
us_data_set |> CSV.write("us_data_set.csv",delim=',',writeheader=true)

"us_data_set.csv"

In [5]:
dropmissing(us_data_set) |> CSV.write("us_data_set_m.csv",delim=',',writeheader=true)

"us_data_set_m.csv"